# Excercises 
# 1. Tune the network
Run the experiment below, explore the different parameters (see suggestions below) and study the result with tensorboard. 
Make a single page (1 a4) report of your findings. Use your visualisation skills to communicate your most important findings.

In [1]:
from mads_datasets import DatasetFactoryProvider, DatasetType

from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics

import torch.optim as optim
from torch import nn
from tomlserializer import TOMLSerializer

We will be using `tomlserializer` to easily keep track of our experiments, and to easily save the different things we did during our experiments.
It can export things like settings and models to a simple `toml` file, which can be easily shared, checked and modified.

First, we need the data. 

In [2]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2025-09-20 16:43:17.058 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\tycoh\.cache\mads_datasets\fashionmnist
2025-09-20 16:43:17.058 | INFO     | mads_datasets.base:download_data:124 - File already exists at C:\Users\tycoh\.cache\mads_datasets\fashionmnist\fashionmnist.pt


We need a way to determine how well our model is performing. We will use accuracy as a metric.

In [3]:
accuracy = metrics.Accuracy()

You can set up a single experiment.

- We will show the model batches of 64 images, 
- and for every epoch we will show the model 100 batches (trainsteps=100).
- then, we will test how well the model is doing on unseen data (teststeps=100).
- we will report our results during training to tensorboard, and report all configuration to a toml file.
- we will log the results into a directory called "modellogs", but you could change this to whatever you want.

In [ ]:
import torch
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps= 100,
    valid_steps= 100,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)


We will use a very basic model: a model with three linear layers.

In [29]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(
    num_classes=10, units1=256, units2=256)

I developped the `tomlserializer` package, it is a useful tool to save configs, models and settings as a tomlfile; that way it is easy to track what you changed during your experiments.

This package will 1. check if there is a `__dict__` attribute available, and if so, it will use that to extract the parameters that do not start with an underscore, like this:

In [6]:
{k: v for k, v in model.__dict__.items() if not k.startswith("_")}

{'training': True, 'num_classes': 10, 'units1': 256, 'units2': 256}

This means that if you want to add more parameters to the `.toml` file, eg `units3`, you can add them to the class like this:

```python
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int, units3: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.units3 = units3  # <-- add this line
```

And then it will be added to the `.toml` file. Check the result for yourself by using the `.save()` method of the `TomlSerializer` class like this:

In [19]:
tomlserializer = TOMLSerializer()
tomlserializer.save(settings, "settings.toml")
tomlserializer.save(model, "model.toml")

Check the `settings.toml` and `model.toml` files to see what is in there.

You can use the `Trainer` class from my `mltrainer` module to train your model. It has the TOMLserializer integrated, so it will automatically save the settings and model to a toml file if you have added `TOML` as a reporttype in the settings.

In [8]:
trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
)
trainer.loop()

2025-09-20 16:43:17.187 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\20250920-164317
2025-09-20 16:43:18.558 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 100/100 [00:00<00:00, 228.06it/s]
2025-09-20 16:43:19.366 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.9176 test 0.6248 metric ['0.7789']
100%|██████████| 100/100 [00:00<00:00, 172.12it/s]
2025-09-20 16:43:20.132 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.5804 test 0.5523 metric ['0.7948']
100%|██████████| 100/100 [00:00<00:00, 254.80it/s]
2025-09-20 16:43:20.711 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.5135 test 0.5307 metric ['0.8086']
100%|██████████| 3/3 [00:02<00:00,  1.47it/s]


Now, check in the modellogs directory the results of your experiment.

We can now loop this with a naive approach, called a grid-search (why do you think i call it naive?).

> because it doens't know what the best setting is it tries al possible combinations

In [42]:
import random

units = [256, 128, 64]
for _ in range(5):  
    unit1 = random.choice(units)
    unit2 = random.choice(units)
    print(f"Random Units: {unit1}, {unit2}")


Random Units: 64, 256
Random Units: 128, 128
Random Units: 128, 256
Random Units: 256, 128
Random Units: 64, 256


Of course, this might not be the best way to search for a model; some configurations will be better than others (can you predict up front what will be the best configuration?).

So, feel free to improve upon the gridsearch by adding your own logic.

In [45]:
import torch

units = [1024,512,256]
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=5,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=128,
    valid_steps=128,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)

units = [256, 128, 64]
for _ in range(5):  
    trainstreamer = train.stream()
    validstreamer = valid.stream()
    unit1 = random.choice(units)
    unit2 = random.choice(units)
    print(f"Random Units: {unit1}, {unit2}")
    model = NeuralNetwork(num_classes=10, units1=unit1, units2=unit2)
    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau
    )
    trainer.loop()

2025-09-21 15:31:33.001 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\20250921-153133
2025-09-21 15:31:33.004 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.


Random Units: 256, 64


100%|██████████| 128/128 [00:00<00:00, 134.28it/s]
2025-09-21 15:31:34.444 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.9495 test 0.6434 metric ['0.7637']
100%|██████████| 128/128 [00:01<00:00, 107.38it/s]
2025-09-21 15:31:36.026 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.5664 test 0.5483 metric ['0.8048']
100%|██████████| 128/128 [00:01<00:00, 124.31it/s]
2025-09-21 15:31:37.588 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.5070 test 0.5162 metric ['0.8206']
100%|██████████| 128/128 [00:01<00:00, 105.10it/s]
2025-09-21 15:31:39.351 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.4791 test 0.4813 metric ['0.8330']
100%|██████████| 128/128 [00:01<00:00, 92.87it/s]
2025-09-21 15:31:41.234 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 0.4700 test 0.4544 metric ['0.8400']
100%|██████████| 5/5 [00:08<00:00,  1.64s/it]
2025-09-21 15:31:41.237 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\202509

Random Units: 64, 128


100%|██████████| 128/128 [00:01<00:00, 123.62it/s]
2025-09-21 15:31:42.700 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.0491 test 0.6903 metric ['0.7509']
100%|██████████| 128/128 [00:00<00:00, 136.13it/s]
2025-09-21 15:31:44.053 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.6088 test 0.5855 metric ['0.7847']
100%|██████████| 128/128 [00:01<00:00, 103.70it/s]
2025-09-21 15:31:45.799 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.5226 test 0.5443 metric ['0.8052']
100%|██████████| 128/128 [00:01<00:00, 97.88it/s]
2025-09-21 15:31:47.576 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.4917 test 0.5064 metric ['0.8240']
100%|██████████| 128/128 [00:01<00:00, 120.47it/s]
2025-09-21 15:31:49.149 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 0.4703 test 0.4982 metric ['0.8119']
100%|██████████| 5/5 [00:07<00:00,  1.58s/it]
2025-09-21 15:31:49.153 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\202509

...

100%|██████████| 128/128 [00:01<00:00, 108.10it/s]
2025-09-21 15:31:50.835 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.0976 test 0.6966 metric ['0.7430']
100%|██████████| 128/128 [00:01<00:00, 111.35it/s]
2025-09-21 15:31:52.394 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.6425 test 0.5965 metric ['0.7793']
100%|██████████| 128/128 [00:01<00:00, 121.31it/s]
2025-09-21 15:31:53.837 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.5461 test 0.5295 metric ['0.8170']
100%|██████████| 128/128 [00:00<00:00, 145.35it/s]
2025-09-21 15:31:55.214 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.5155 test 0.5278 metric ['0.8163']
100%|██████████| 128/128 [00:01<00:00, 108.50it/s]
2025-09-21 15:31:56.836 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 0.4694 test 0.5375 metric ['0.8070']
2025-09-21 15:31:56.839 | INFO     | mltrainer.trainer:__call__:252 - best loss: 0.5278, current loss 0.5375.Counter 1/10.
100%|██████████| 5/5 [00:07

Random Units: 64, 64


100%|██████████| 128/128 [00:01<00:00, 121.92it/s]
2025-09-21 15:31:58.443 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.1001 test 0.7259 metric ['0.7223']
100%|██████████| 128/128 [00:01<00:00, 104.92it/s]
2025-09-21 15:32:00.033 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.6333 test 0.6062 metric ['0.7875']
100%|██████████| 128/128 [00:01<00:00, 124.73it/s]
2025-09-21 15:32:01.543 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.5478 test 0.5392 metric ['0.8027']
100%|██████████| 128/128 [00:01<00:00, 104.20it/s]
2025-09-21 15:32:03.267 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.4991 test 0.5219 metric ['0.8136']
100%|██████████| 128/128 [00:01<00:00, 108.37it/s]
2025-09-21 15:32:04.955 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 0.5008 test 0.4850 metric ['0.8298']
100%|██████████| 5/5 [00:08<00:00,  1.62s/it]
2025-09-21 15:32:04.962 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\20250

Random Units: 64, 64


100%|██████████| 128/128 [00:01<00:00, 123.02it/s]
2025-09-21 15:32:06.376 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.0792 test 0.7149 metric ['0.7416']
100%|██████████| 128/128 [00:00<00:00, 130.63it/s]
2025-09-21 15:32:07.796 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.6204 test 0.6702 metric ['0.7667']
100%|██████████| 128/128 [00:00<00:00, 130.72it/s]
2025-09-21 15:32:09.213 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.5358 test 0.5376 metric ['0.8011']
100%|██████████| 128/128 [00:01<00:00, 121.18it/s]
2025-09-21 15:32:10.648 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.5023 test 0.5562 metric ['0.7966']
2025-09-21 15:32:10.648 | INFO     | mltrainer.trainer:__call__:252 - best loss: 0.5376, current loss 0.5562.Counter 1/10.
100%|██████████| 128/128 [00:01<00:00, 112.09it/s]
2025-09-21 15:32:12.205 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 0.4843 test 0.5121 metric ['0.8159']
100%|██████████| 5/5 [00:07